# Pre-processing and training LDA

The purpose of this tutorial is to show you how to pre-process text data, and how to train the LDA model on that data. 

I would also encourage you to consider each step when applying the model to your data, instead of just blindly applying my solution. The different steps will depend on your data and possibly your goal with the model.

In the following sections, we will go through pre-processing the data and training the model.

In this tutorial we will:
    1. Load data.
    2. Pre-process data.
    3. Transform documents to a vectorized form.
    4. Train an LDA model.

If you are not familiar with the LDA model or how to use it in Gensim, I suggest you read up on that before continuing with this tutorial. Basic understanding of the LDA model should suffice. Examples:
    1. Gentle introduction to the LDA model: http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/
    2. Gensim's LDA API documentation: https://radimrehurek.com/gensim/models/ldamodel.html
    3. Topic modelling in Gensim: http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

## 1. Data
We will be using some papers from the NIPS (Neural Information Processing Systems) conference. NIPS is a machine learning conference so the subject matter should be well suited for most of the target audience of this tutorial.

You can download the data from Sam Roweis' website (http://www.cs.nyu.edu/~roweis/data.html).

Note that the corpus contains 1740 documents, and not particularly long ones. So keep in mind that this tutorial is not geared towards efficiency, and be careful before applying the code to a large dataset.

Below we are simply reading the data.

In [1]:
# Read data.

import os

# Folder containing all NIPS papers.
data_dir = './nipstxt/'

# Folders containin individual NIPS papers.
yrs = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
dirs = ['nips' + yr for yr in yrs]

# Read all texts into a list.
docs = []
for yr_dir in dirs:
    files = os.listdir(data_dir + yr_dir)
    for filen in files:
        # Note: ignoring characters that cause encoding errors.
        with open(data_dir + yr_dir + '/' + filen) as fid:
            txt = fid.read()
        docs.append(txt)

## 2. Pre-process and vectorize the documents
Among other things, we will:
1. Split the documents into tokens.
2. Lemmatize the tokens.
3. Compute bigrams.
4. Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We remove numeric tokens and tokens that are only a single character, as they don't tend to be useful, and the dataset contains a lot of them.

In [2]:
# Tokenize the documents.

from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isdigit()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a stemmer in this case because it produces more readable words. Output that is easy to read is very desirable in topic modelling.

In [5]:
# Lemmatize the documents.

from nltk.stem.wordnet import WordNetLemmatizer

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
lem_docs = []
for doc in docs:
    try:
        lem_docs.append([lemmatizer.lemmatize(token) for token in doc])
    except:
        pass

We find bigrams in the documents. Bigrams are sets of two adjacent words. Using bigrams we can get phrases like "machine_learning" in our output (spaces are replaced with underscores); without bigrams we would only get "machine" and "learning".

Note that in the code below, we find bigrams and then add them to the original data, because we would like to keep the words "machine" and "learning" as well as the bigram "machine_learning".

Note that computing n-grams of large dataset can be very computationally intentensive and memory intensive.

In [8]:
# Compute bigrams.

from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(lem_docs, min_count=20)
for idx in range(len(lem_docs)):
    for token in bigram[lem_docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            lem_docs[idx].append(token)

/Users/jiozhan/anaconda3/envs/py27/lib/python2.7/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


We remove rare words and common words based on their document frequency. Below we remove words that appear in less than 20 documents or in more than 50% of the documents. Consider trying to remove words only based on their frequency, or maybe combining that with this approach.

In [10]:
# Remove rare and common tokens.

from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(lem_docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)


Finally, we transform the documents to a vectorized form. We simply compute the frequency of each word, including the bigrams.


In [12]:
# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in lem_docs]

Let's see how many tokens and documents we have to train on.

In [14]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5234
Number of documents: 861


## Training

We are ready to train the LDA model. We will first discuss how to set some of the training parameters.

First of all, the elephant in the room: how many topics do I need? There is really no easy answer for this, it will depend on both your data and your application. I have used 10 topics here because I wanted to have a few topics that I could interpret and "label", and because that turned out to give me reasonably good results. You might not need to interpret all your topics, so you could use a large number of topics, for example 100.

The chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. I've set chunksize = 2000, which is more than the amount of documents, so I process all the data in one go. Chunksize can however influence the quality of the model, as discussed in Hoffman and co-authors [2], but the difference was not substantial in this case.

passes controls how often we train the model on the entire corpus. Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable logging (as described in many Gensim tutorials), and set eval_every = 1 in LdaModel. When training the model look for a line in the log that looks something like this:
    
    2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set passes = 20 you will see this line 20 times. Make sure that by the final passes, most of the documents have converged. So you want to choose both passes and iterations to be high enough for this to happen.

We set alpha = 'auto' and eta = 'auto'. Again this is somewhat technical, but essentially we are automatically learning two parameters in the model that we usually would have to specify explicitly.

In [15]:
# Train LDA model.

from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 1min 28s, sys: 269 ms, total: 1min 28s
Wall time: 1min 28s


We can compute the topic coherence of each topic. Below we display the average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see docs, https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel.top_topics), Gensim has recently obtained an implementation of the "AKSW" topic coherence measure (see accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can see that the topics below make a lot of sense. However, they are not without flaws. We can see that there is substantial overlap between some topics, others are hard to interpret, and most of them have at least some terms that seem out of place. If you were able to do better, feel free to share your methods on the blog at http://rare-technologies.com/lda-training-tips/ !

In [16]:
top_topics = model.top_topics(corpus, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

TypeError: only integer scalar arrays can be converted to a scalar index

In [17]:
model.show_topics()

[(0,
  u'0.023*"image" + 0.011*"visual" + 0.009*"motion" + 0.007*"filter" + 0.006*"map" + 0.006*"response" + 0.006*"object" + 0.006*"pixel" + 0.006*"field" + 0.005*"source"'),
 (1,
  u'0.020*"neuron" + 0.014*"cell" + 0.011*"circuit" + 0.011*"chip" + 0.008*"analog" + 0.008*"voltage" + 0.006*"synaptic" + 0.006*"response" + 0.005*"synapsis" + 0.005*"synapse"'),
 (2,
  u'0.009*"classifier" + 0.008*"probability" + 0.006*"gaussian" + 0.006*"estimate" + 0.006*"mixture" + 0.006*"class" + 0.005*"sample" + 0.005*"likelihood" + 0.005*"kernel" + 0.004*"density"'),
 (3,
  u'0.013*"hidden" + 0.010*"layer" + 0.008*"sequence" + 0.008*"net" + 0.008*"node" + 0.006*"hidden_unit" + 0.005*"prediction" + 0.005*"language" + 0.005*"back" + 0.005*"propagation"'),
 (4,
  u'0.007*"generalization" + 0.007*"net" + 0.007*"memory" + 0.006*"threshold" + 0.006*"hidden" + 0.005*"hidden_unit" + 0.004*"layer" + 0.004*"bound" + 0.004*"capacity" + 0.004*"theorem"'),
 (5,
  u'0.007*"node" + 0.006*"equation" + 0.006*"matrix"

In [19]:
model.top_topics(corpus)

TypeError: only integer scalar arrays can be converted to a scalar index